In [39]:
#Script para enviar tabela xlsx para SQL da CIDADE

import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text

conexaoDB = ('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-HV85ADA\\PROJETO_MERCADO;DATABASE=MERCADO_OLTP;Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conexaoDB}')

caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\BANCO DE DADOS\\DADOS\\CIDADE\\"
arquivo = r"CIDADES.csv"
caminhocompleto = caminho+arquivo

df_cidades = pd.read_csv(caminhocompleto,delimiter=";")

estados_brasileiros = ["Acre","Alagoas","Amazonas","Amapa","Bahia","Ceara","Distrito Federal","Espirito Santo",
"Goias","Maranhao","Minas Gerais","Mato Grosso do Sul","Mato Grosso","Para","Paraiba","Pernambuco","Piaui",
"Parana","Rio de Janeiro","Rio Grande do Norte","Rondonia","Roraima","Rio Grande do Sul","Santa Catarina",
"Sergipe","Sao Paulo","Tocantins"]

siglas_estados = {
    "Acre": "AC",
    "Alagoas": "AL",
    "Amapa": "AP",
    "Amazonas": "AM",
    "Bahia": "BA",
    "Ceara": "CE",
    "Distrito Federal": "DF",
    "Espirito Santo": "ES",
    "Goias": "GO",
    "Maranhao": "MA",
    "Mato Grosso": "MT",
    "Mato Grosso do Sul": "MS",
    "Minas Gerais": "MG",
    "Para": "PA",
    "Paraiba": "PB",
    "Parana": "PR",
    "Pernambuco": "PE",
    "Piaui": "PI",
    "Rio de Janeiro": "RJ",
    "Rio Grande do Norte": "RN",
    "Rio Grande do Sul": "RS",
    "Rondonia": "RO",
    "Roraima": "RR",
    "Santa Catarina": "SC",
    "Sao Paulo": "SP",
    "Sergipe": "SE",
    "Tocantins": "TO"
}

regioes = {
    "Norte":        ["Acre", "Amapa", "Amazonas", "Para", "Rondonia", "Roraima", "Tocantins"],
    "Nordeste":     ["Alagoas", "Bahia", "Ceara", "Maranhao", "Paraiba", "Pernambuco", "Piaui", "Rio Grande do Norte", "Sergipe"],
    "Centro-Oeste": ["Distrito Federal", "Goias", "Mato Grosso", "Mato Grosso do Sul"],
    "Sudeste":      ["Espirito Santo", "Minas Gerais", "Rio de Janeiro", "Sao Paulo"],
    "Sul":          ["Parana", "Rio Grande do Sul", "Santa Catarina"]
}

df_EST = []

for estado in estados_brasileiros:
    for regiao, estados_regiao in regioes.items():
        if estado in estados_regiao:
            df_EST.append({
                "Nome":   estado,
                "UF":     siglas_estados[estado],
                "Regiao": regiao,
                "Pais":   "Brasil"
            })
            break

df_EST = pd.DataFrame(df_EST)

estados = df_cidades["COD_UF"].unique().tolist()
estados.sort()

id_estado = [indice + 1 for indice in range(len(estados))]

df_estados = pd.DataFrame({
                    'ESTADO':    estados,
                    'ID_ESTADO': id_estado,

})

mapeamento = dict(zip(df_estados['ESTADO'], df_estados['ID_ESTADO']))

df_cidades['ID_ESTADO'] = df_cidades['COD_UF'].map(mapeamento)



df_CIDADE = pd.DataFrame({
                'NOME':         df_cidades['NOME_MUN'],
                'ID_ESTADO':    df_cidades['ID_ESTADO'],

})

df_ESTADO = pd.DataFrame({
                'ESTADO':       df_EST['Nome'],
                'UF':           df_estados['ESTADO'],
                'REGIAO':       df_EST['Regiao'],
                'PAIS':         df_EST['Pais']

})

display(df_CIDADE)
display(df_ESTADO)

tipo_dados_cidade = {

    'NOME':         VARCHAR(50),
    'ID_ESTADO':    INTEGER

}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE CIDADE"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_CIDADE.to_sql(name='CIDADE',con=engine,if_exists='append',index=False,dtype=tipo_dados_cidade)

tipo_dados_estado = {

    'ESTADO':  VARCHAR(25),
    'UF':      VARCHAR(2),
    'REGIAO':  VARCHAR(20),
    'PAIS':    VARCHAR(20),

}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE ESTADO"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_ESTADO.to_sql(name='ESTADO',con=engine,if_exists='append',index=False,dtype=tipo_dados_estado)

engine.dispose()

,NOME,ID_ESTADO
0,Alta Floresta D Oeste,21
1,Ariquemes,21
2,Cabixi,21
3,Cacoal,21
4,Cerejeiras,21
...,...,...
5559,Vianópolis,9
5560,Vicentinópolis,9
5561,Vila Boa,9
5562,Vila Propício,9


,ESTADO,UF,REGIAO,PAIS
0,Acre,AC,Norte,Brasil
1,Alagoas,AL,Nordeste,Brasil
2,Amazonas,AM,Norte,Brasil
3,Amapa,AP,Norte,Brasil
4,Bahia,BA,Nordeste,Brasil
5,Ceara,CE,Nordeste,Brasil
6,Distrito Federal,DF,Centro-Oeste,Brasil
7,Espirito Santo,ES,Sudeste,Brasil
8,Goias,GO,Centro-Oeste,Brasil
9,Maranhao,MA,Nordeste,Brasil
